# Indepth Example

This is a more in depth example showing:
1. Calling the [`market-data/reference-data/v3/search`](https://developer.platts.com/servicecatalog#/MarketData/v3/Reference%20Data) to find out which MDC (Market Data Category) I have access to.
2. Calling the [`market-data/v3/value/current/mdc`](https://developer.platts.com/servicecatalog#/MarketData/v3/Market%20Data) endpoint with those MDCs to get the latest assessments for the symbols in those MDCs
3. Pivoting the results

## Fetching Data

### Install Dependencies

```{tip}
`pip install requests pandas pprint`
```

Import dependencies

In [1]:
import requests
import pandas as pd
from pprint import pprint

Set `username`, `password`, and `apikey`

In [2]:
username = "USERNAME"
password = "PASSWORD"
apikey = "APIKey"

```{seealso}
This `get_token` function is taken from [Generating a Token](../token.ipynb)
```

In [3]:
def get_token(username, password, apikey):
  body = {
    "username": username,
    "password": password
  }
  headers = {
    "appkey": apikey
  }
  try:
    r = requests.post("https://api.platts.com/auth/api", data=body, headers=headers)
    r.raise_for_status()
    return r.json()["access_token"]
  except Exception as err:
    if r.status_code >= 500:
      print(err)
    else:
      print(r.status_code, r.json())

### Get MDC List
1. Create a function - `get_mdc_list` - to call the Platts Reference Data Search Endpoint
2. Requires a `username`, `password` and `apikey`
3. Returns a `dict` (json) of MDC's that you are subscribed to as well as the count of symbols within that MDC

In [4]:
def get_mdc_list(username, password, apikey):
  token = get_token(username, password, apikey)

  params = {
    "Facet.Field": "mdc",
    "PageSize": 1,
    "Field": "symbol",
    "subscribed_only": True
  }
  headers = {
    "Authorization": f"Bearer {token}",
    "appkey": apikey
  }

  try:
    r = requests.get("https://api.platts.com/market-data/reference-data/v3/search",
      params=params, headers=headers)
    r.raise_for_status()
    return r.json()
  except Exception as err:
    if r.status_code >= 500:
      print (err)
    else:
      print(r.status_code, r.json())
    raise



Invoke `get_mdc_list` and print truncated results

In [5]:
mdc_data = get_mdc_list(username, password, apikey)
pprint(list(mdc_data['facets']['facet_counts']['mdc'].items())[:10])

[('IO', '22003'),
 ('TE', '821'),
 ('EM', '278'),
 ('NK', '36'),
 ('YN', '30'),
 ('SG', '83'),
 ('CD', '349'),
 ('BA', '430'),
 ('TK', '262'),
 ('PN', '188')]


### Get Current Assesments by MDC
1. Create a function - `get_current_assesments_by_mdc` - to call the Platts Current/MDC Endpoint
2. Requires a `username`, `password`, `apikey` and an `mdc` code
3. Returns a `dict` (json) of assessment data from the Platts API

In [6]:
def get_current_assessments_by_mdc(username, password, apikey, mdc):
  token = get_token(username, password, apikey)

  # must be quotes around mdc
  params = {
    "filter": f'mdc: "{mdc}"',
    "pagesize": 10000
  }

  headers = {
    "Authorization": f"Bearer {token}",
    "appkey": apikey
  }

  try:
    r = requests.get("https://api.platts.com/market-data/v3/value/current/mdc", 
      params=params, headers=headers)
    r.raise_for_status()
    return r.json()
  except Exception as err:
    if r.status_code >= 500:
      print (err)
    else:
      print(r.status_code, r.json())
    raise

Invoke `get_current_assessments_by_mdc` and print truncated results

In [7]:
mdc = list(mdc_data['facets']['facet_counts']['mdc'])[1]
print(f"Fetching data for mdc: {mdc}")
data = get_current_assessments_by_mdc(username, password, apikey, mdc)
pprint(data["results"][:5])

Fetching data for mdc: TE
[{'data': [{'assessDate': '2022-06-10T00:00:00',
            'bate': 'w',
            'isCorrected': 'N',
            'modDate': '2022-06-10T20:05:42',
            'value': 123565}],
  'symbol': 'NMHO000'},
 {'data': [{'assessDate': '2022-06-10T00:00:00',
            'bate': 'c',
            'isCorrected': 'N',
            'modDate': '2022-06-10T20:05:42',
            'value': 4.3667},
           {'assessDate': '2022-06-10T00:00:00',
            'bate': 'e',
            'isCorrected': 'N',
            'modDate': '2022-06-10T20:05:42',
            'value': 49760},
           {'assessDate': '2022-06-10T00:00:00',
            'bate': 'h',
            'isCorrected': 'N',
            'modDate': '2022-06-10T20:05:42',
            'value': 4.5135},
           {'assessDate': '2022-06-10T00:00:00',
            'bate': 'l',
            'isCorrected': 'N',
            'modDate': '2022-06-10T20:05:42',
            'value': 4.3155},
           {'assessDate': '2022-06-10T00

```{note}
Every response contains a `metadata` object. This is useful for paging. If the `count` is larger than the `pageSize` than you will have to make multiple API calls in order to get all records
```

In [8]:
print(data['metadata'])

{'count': 2088, 'pageSize': 10000, 'page': 1, 'totalPages': 1, 'queryTime': '72 ms'}


## Using Pandas

### Creating a DataFrame
Flattening our JSON response into a DataFrame

In [9]:
df = pd.json_normalize(data['results'], record_path=['data'], meta="symbol")
# Parse dates so that we can use datetime operations
cols = ["assessDate", "modDate"]
df[cols] = df[cols].apply(pd.to_datetime)
df

,bate,assessDate,value,isCorrected,modDate,symbol
0,w,2022-06-10,123565.0000,N,2022-06-10 20:05:42,NMHO000
1,c,2022-06-10,4.3667,N,2022-06-10 20:05:42,NMHO001
2,e,2022-06-10,49760.0000,N,2022-06-10 20:05:42,NMHO001
3,h,2022-06-10,4.5135,N,2022-06-10 20:05:42,NMHO001
4,l,2022-06-10,4.3155,N,2022-06-10 20:05:42,NMHO001
...,...,...,...,...,...,...
2083,o,2022-06-10,2.8017,N,2022-06-11 00:03:51,XNHOZ24
2084,c,2022-06-10,2.7232,N,2022-06-11 00:03:51,XNHOZ25
2085,h,2022-06-10,2.7232,N,2022-06-11 00:03:51,XNHOZ25
2086,l,2022-06-10,2.7232,N,2022-06-11 00:03:51,XNHOZ25


### Pivoting the Data by Bate Code

In [10]:
pivoted = df.pivot(index=['symbol', 'assessDate'], columns=['bate'], values=['value'])
pivoted

value                                           
bate                     c        e       h       l       o         w
symbol  assessDate                                                   
NMHO000 2022-06-10     NaN      NaN     NaN     NaN     NaN  123565.0
NMHO001 2022-06-10  4.3667  49760.0  4.5135  4.3155  4.4901   27949.0
NMHO002 2022-06-10  4.2582  51204.0  4.3993  4.2127  4.3780   27950.0
NMHO003 2022-06-10  4.1739  42171.0  4.3074  4.1277  4.2845   21025.0
NMHO004 2022-06-10  4.0949  20742.0  4.2162  4.0490  4.1945   10253.0
...                    ...      ...     ...     ...     ...       ...
XNHOZ21 2021-11-30  2.0638      NaN  2.0638  2.0638  2.0638       NaN
XNHOZ22 2022-06-10  3.9342      NaN  3.9342  3.9342  3.9342       NaN
XNHOZ23 2022-06-10  3.0799      NaN  3.0799  3.0799  3.0799       NaN
XNHOZ24 2022-06-10  2.8017      NaN  2.8017  2.8017  2.8017       NaN
XNHOZ25 2022-06-10  2.7232      NaN  2.7232  2.7232  2.7232       NaN

[534 rows x 6 columns]

### Flattening the Results

In [11]:
pivoted_df = pd.DataFrame(pivoted.to_records())
sorted_df = pivoted_df.sort_values(by=['symbol'], ascending=[True])
pivoted_df.head(25)

,symbol,assessDate,"('value', 'c')","('value', 'e')","('value', 'h')","('value', 'l')","('value', 'o')","('value', 'w')"
0,NMHO000,2022-06-10,NaN,NaN,NaN,NaN,NaN,123565.0
1,NMHO001,2022-06-10,4.3667,49760.0,4.5135,4.3155,4.4901,27949.0
2,NMHO002,2022-06-10,4.2582,51204.0,4.3993,4.2127,4.3780,27950.0
3,NMHO003,2022-06-10,4.1739,42171.0,4.3074,4.1277,4.2845,21025.0
4,NMHO004,2022-06-10,4.0949,20742.0,4.2162,4.0490,4.1945,10253.0
5,NMHO005,2022-06-10,4.0157,15520.0,4.1157,3.9711,4.1053,6924.0
6,NMHO006,2022-06-10,3.9342,28598.0,4.0350,3.8781,4.0081,10777.0
7,XCLY000,2022-06-09,NaN,761.0,NaN,NaN,NaN,0.0
8,XCLY001,2022-06-10,4.3137,NaN,4.3137,4.3137,4.3137,NaN
9,XCLY002,2022-06-10,4.2307,NaN,4.2307,4.2307,4.2307,NaN
